# End Use Profiles to Alfalfa 

This notebook sets up the workflow to download models from the comsstock/resstock end use load profiles database, and creates folders that can be uploaded to Alfalfa.

In [15]:
pip install https://github.com/NREL/alfalfa.git@buildstock

Note: you may need to restart the kernel to use updated packages.


  ERROR: HTTP error 404 while getting https://github.com/NREL/alfalfa.git@buildstock
ERROR: Could not install requirement https://github.com/NREL/alfalfa.git@buildstock because of HTTP error 404 Client Error: Not Found for url: https://github.com/NREL/alfalfa.git@buildstock for URL https://github.com/NREL/alfalfa.git@buildstock


In [16]:
import os
import shutil
import json
import pandas
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient
from pprint import pprint
from pathlib import Path

### Define alfalfa client object


In [17]:
ac = AlfalfaClient(host='https://alfalfa-staging.nrel.gov')

### Defining inputs
#### Link to building models for comstock_tmy3_release_1
https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=nrel-pds-building-stock%2Fend-use-load-profiles-for-us-building-stock%2F2021%2Fcomstock_tmy3_release_1%2Fbuilding_energy_models%2F
#### Link to building models for resstock_tmy3_release_1
https://data.openei.org/s3_viewer?bucket=oedi-data-lake&prefix=nrel-pds-building-stock%2Fend-use-load-profiles-for-us-building-stock%2F2021%2Fresstock_tmy3_release_1%2Fbuilding_energy_models%2F

In [19]:
id = ['bldg0000001','bldg0000002']
year = '2021'
dataset_name = 'comstock_tmy3_release_1'
upgrade = '00'
# Folder name to download models
folder = 'comstock'

### Download models and weather file from buildstock to previously defined local folder

In [21]:
objA = Inputs
objA.__init__(Inputs, year, dataset_name, upgrade, id,  folder_name)

objB = Models
objB.download_models(objA)

ModuleNotFoundError: No module named 'alfalfa'

### Upload models to Alfalfa

Submit models to Alfalfa server


In [22]:
files = list(item for item in folder.iterdir() if item.is_dir())
print(f"uploading {files}")
model_ids = ac.submit(files)
print(model_ids)


AttributeError: 'str' object has no attribute 'iterdir'

### Define Alfalfa simulation parameters

- start_dt : This is the start date and time for the simulation in the following format (YYYY, M, D, H, M, S)
- end_dt : This is the end date and time for the simulation in the following format (YYYY, M, D, H, M, S)
- params : This defines the simulation parameters for the simulation including the start/end date and time and whether the model advances via API calls or a specified timescale
    

In [23]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2023, 5, 10, 12, 2, 0)
end_dt = datetime.datetime(2023, 5, 10, 12, 3, 0)

# For external_clock == true, API calls are used to advance the model.
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": True,
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

### Start simulations 

Start simulations as per the parameters defined

In [ ]:
# Start Simulations
print(f"Starting site: {model_ids}")
ac.start(model_ids, **params)
print('Done')

### Set the model ID 

In [28]:
# set model ID
model_id = '246077f0-566d-11ee-80a8-f546b406f534'

### Get the model's input points
Get a list of all of the model's input points (EP Actuators)
- model_id - the id of the site returned by the `ac.submit` function

In [ ]:
print(f"{model_id} inputs:")
pprint(ac.get_inputs(model_id))

### Set model input point

To set an input value use the `ac.set_inputs(model_id, inputs)` function.

- `inputs_dict` - a dictionary of input names and the desired values

In [ ]:
input_dict = {'Zone_PVAV_with_PFP_Boxes_and_Reheat_1_Outside_Air_Damper_CMD': 10}
ac.set_inputs(model_id, input_dict)
print('Done')

### Advance the model
12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.

In [ ]:
timesteps = 5
for _ in range(timesteps):
    ac.advance([model_id])
    print(f"Model advanced to time: {ac.get_sim_time(model_id)}")

print('Done')

### Get model's outputs
Query the outputs (EP Sensors) of the models as well as their values

In [ ]:
print(f"{model_id} outputs:")

# Print Simulation Time
pprint(ac.get_sim_time(model_id))

# Print output values
pprint(ac.get_outputs(model_id))

### Stop the simulations

In [ ]:
ac.stop(model_id)